In [13]:
# import libraries, packages
import numpy as np
import matplotlib.pyplot as plt
import scipy
import math
import scipy.optimize

In [8]:
# borrower object
# each borrower has a group membership, group creditworthiness, individual signal (theta)

class Borrower:
  def __init__(self, group, pi, theta):
    self.group = group
    # clean up constructor so assignment is easier for group membership      
    self.pi = pi
    self.theta = theta

In [66]:
#parameters

q = []
# set for group u
u = []

# benefit of getting a loan to a borrower
benefit = 10

x_q = 0.75 # net return to lender if qualified borrower gets loan, q = qualified
x_u = -0.25 # net loss to lender if unqualified borrower gets loan, u = unqualified

ratio = abs(x_q / x_u) # ratio of net gain to loss

# lender's beliefs about group probabilities of repayment
pi_q = 0.2 
pi_u = 0.1

# change in lender's perceptions? (how creditworthiness improves)

# INCOMPLETE cost to become creditworthy. Cost varies for each member but is the same distribution for the both groups
cost = 0

# INCOMPLETE fraction of workers that choose to become creditworthy G(c)

Test space

In [74]:
tester_q = [Borrower(True, pi_q, 0.5),Borrower(True, pi_q, 0.7), Borrower(True, pi_q, 1)]
tester_u = [Borrower(False, pi_u, 0),Borrower(False, pi_u, 0.2), Borrower(False, pi_u, 0.4)]

signal_pdf(tester_u, 0)
signal_pdf(tester_q, 0)

likelihood_ratio(tester_q, tester_u, 0)



3.0

Functions

In [78]:

# probability signal does not exceed theta in F_q(theta) or F)q(theta)
def signal_probability(q, given_theta):

    if not 0 <= given_theta <= 1:
        raise ValueError("Threshold must be between 0 and 1")

    count = sum(1 for individual in q if individual.theta <= given_theta)
    probability = count / len(q) if q else 0.0

    return probability


# density functions for signal probability
# helper function to generate the pdf
def generate_pdf(group):
    total_borrowers = len(group)
    if total_borrowers == 0:
        raise ValueError("The borrowers list is empty")
    

    theta_counts = {}
    for borrower in group:
        theta = borrower.theta
        if theta in theta_counts:
            theta_counts[theta] += 1
        else:
            theta_counts[theta] = 1

    # Convert counts to probabilities
    pdf = {theta: count / total_borrowers for theta, count in theta_counts.items()}

    return pdf

# gives the specific pdf value of a group given theta
def signal_pdf(group, given_theta):
    if not 0 <= given_theta <= 1:
        raise ValueError("given_theta must be between 0 and 1")

    # Generate the PDF
    pdf = generate_pdf(group)

    # Sum the probabilities for theta values less than x
    sum_prob = sum(prob for theta, prob in pdf.items() if theta < given_theta)

    return sum_prob

# liklihood ratio at a given theta
def likelihood_ratio(q, u, given_theta):
    q_value = signal_pdf(q, given_theta)

    if q_value == 0:
        return 1

    return signal_pdf(u, given_theta) / q_value
    
# Lender's posterior probability borrower is qualified
def qualified_prob(borrower, q, u):
    pi = borrower.pi
    theta = borrower.theta

    result = 1 / (1+((1-pi/pi)) * likelihood_ratio(q, u, theta))

# Lender's expected payoff from lending to anyone
def payoff(borrower, q, u):
    return qualified_prob(borrower, q, u) * x_q - (1-qualified_prob(borrower, q, u)) * x_u


# lender lends iff
def employer_assignment(x_q, x_u, q, u, given_theta):
    ratio = x_q / x_u

    return ratio >= (1-likelihood_ratio(q, u, given_theta))/(likelihood_ratio(q, u, given_theta))


# how employer determines the standard s*
# minimum value of theta such that r > [(1-pi/pi)*liklihood]

def standard(q, u, pi, ratio, theta_bounds=(0, 1), tol=1e-6):
    """
    Finds the minimum value of theta such that the given inequality holds.

    Parameters:
    pi (float): The given pi value.
    ratio (float): The ratio to compare against.
    theta_bounds (tuple): A tuple of (min_theta, max_theta) representing the bounds within which to search.
    tol (float): The tolerance for the numerical solver.

    Returns:
    float: The minimum value of theta satisfying the inequality.
    """
    if not 0 <= pi <= 1:
        raise ValueError("pi must be between 0 and 1")
    
    # The value we are comparing against
    compare_value = (1 - pi) / pi

    # Define the function for the root finding
    def inequality(theta):
        return ratio - compare_value * likelihood_ratio(q, u, theta)
    
    # FLAG if there's no solution, then the lender lends to no one, and the credit standard is `100%`
    if np.sign(ratio - compare_value * likelihood_ratio(q, u, 0)) == np.sign(ratio - compare_value * likelihood_ratio(q, u, 1)):
        return 1

    # Use a numerical solver to find the root within the given bounds
    result = scipy.optimize.root_scalar(inequality, bracket=theta_bounds, method='brentq', xtol=tol)

    if not result.converged:
        raise ValueError("Solver did not converge to a solution")

    return result.root
    

0.4999997499999998

In [ ]:
# expected benefit of becoming creditworthy for borrower facing a given standard:

def expected_benefit(q, u, standard):
    return benefit * (signal_probability(u, standard) - signal_probability(q, standard))


# TO DO
# add prob of assignment given group function
# add net payoff function
# code simulation element
# how to factor in increasing credit rating? -- increases Pi_w and pi_b by some amount